![Photo by <a href="https://unsplash.com/@planetvolumes?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Planet Volumes</a> on <a href="https://unsplash.com/photos/awYEQyYdHVE?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>](images/jupiter.jpg){.preview-image .rounded}

This blog post (and the [source notebook](https://github.com/seeM/blog/blob/main/posts/jupyter-server-a-whirlwind-tour.ipynb)) is an executable playground for understanding how to communicate with Jupyter Servers. You can think of it as a barebones Jupyter frontend, since we'll be implementing the full lifecycle including creating a new notebook, writing and executing code cells, and shutting down the server.

I'm [building my own native macOS Jupyter frontend](https://twitter.com/wasimlorgat/status/1611615676220817415?s=20) and writing about my experience and learnings along the way. In order to do that, I need to be familiar with how Jupyter Servers works.

My approach to learning this was a combination of using Chrome dev tools to inspect network requests in Jupyter Lab, and reading the wonderful Jupyter Server docs (particularly the [REST API reference](https://jupyter-server.readthedocs.io/en/latest/developers/rest-api.html)). I'll include links to the relevant docs in each section below.

Let's get started!

## Starting the server

To start, ensure that you're running a Jupyter Server in another process (e.g. in a terminal) by entering the following command:

```sh
jupyter server
```

Once the server is running, update the `url_with_token` variable below to match what's displayed in the terminal output. For example, it should output something like this:

```
[C 2023-01-07 12:03:57.482 ServerApp]

    To access the server, open this file in a browser:
        file:///Users/seem/Library/Jupyter/runtime/jpserver-80287-open.html
    Or copy and paste one of these URLs:
        http://localhost:8889/?token=72b22f0cee26baaa6aed492b6fed5a010d57bd6c0e1adcce
     or http://127.0.0.1:8889/?token=72b22f0cee26baaa6aed492b6fed5a010d57bd6c0e1adcce
```

In [1]:
# NB: Update this based on your terminal output
url_with_token = 'http://localhost:8889/?token=72b22f0cee26baaa6aed492b6fed5a010d57bd6c0e1adcce'

## Authenticating

First, we'll do a quick check that there is a server at the defined `url`. We need to get the URL without the `token` query parameter:

In [2]:
from urllib.parse import urlparse

In [3]:
url = urlparse(url_with_token)._replace(query=None).geturl()
url

'http://localhost:8889/'

Now we can make the request:

In [4]:
import requests

In [5]:
requests.get(url)

<Response [200]>

A 200 response means that the server processed the request successfully.

Next we need to authenticate. What happens if we try to make a request to an endpoint that requires authentication, for example `GET /api/contents`?

In [6]:
requests.get(url + 'api/contents')

<Response [403]>

It fails with `403 Forbidden`.

If we include our token in the `Authorization` header:

In [7]:
token = urlparse(url_with_token).query.split('=')[-1]
headers = {'Authorization': f'token {token}'}
requests.get(url + 'api/contents', headers=headers)

<Response [200]>

... it works!

Let's create a `requests.Session` so we don't have to keep specifying the header:

In [8]:
session = requests.Session()
session.headers.update(headers)

## Managing files

Jupyter Server lets you manage files via the [Contents API](https://jupyter-server.readthedocs.io/en/latest/developers/contents.html). Browser frontends access this via the `/api/contents` [REST API](https://jupyter-server.readthedocs.io/en/latest/developers/rest-api.html).

Let's use the Contents API to create a file, rename it, and write some contents to it.

### List the contents of a directory

In [9]:
#| echo: false
#| output: false
!rm ../temp/*

rm: ../temp/*: No such file or directory


`GET /api/contents/<path>` returns the contents of the file or directory at `path`. You can think of it as `ls` for Jupyter Server:

In [10]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-13T08:22:47.286711Z',
 'created': '2023-01-13T08:22:47.286711Z',
 'content': [],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

Since the directory is currently empty, `content` is an empty list.

### Create an empty notebook

`POST /api/contents/<path>` creates an empty file in the directory at `path`. You can specify the `type` of the file in the request body:

In [11]:
session.post(url + 'api/contents', json={'type': 'notebook'})

<Response [201]>

The 201 status code means that the request succeeded and a resource was created.

Let's confirm that the file exists with `GET /api/contents`:

In [12]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-13T10:15:41.135210Z',
 'created': '2023-01-13T10:15:41.135210Z',
 'content': [{'name': 'Untitled.ipynb',
   'path': 'Untitled.ipynb',
   'last_modified': '2023-01-13T10:15:41.140623Z',
   'created': '2023-01-13T10:15:41.140623Z',
   'content': None,
   'format': None,
   'mimetype': None,
   'size': 72,
   'writable': True,
   'type': 'notebook'}],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

The response is a nested dict. The root dict refers to the root directory as before, however, `content` now contains the newly created notebook named `Untitled.ipynb`.

We can get the contents of this file using the same method but referring to the file's path i.e. `GET /api/contents/<path>`:

In [13]:
data = session.get(url + 'api/contents/Untitled.ipynb').json()
data

{'name': 'Untitled.ipynb',
 'path': 'Untitled.ipynb',
 'last_modified': '2023-01-13T10:15:41.140623Z',
 'created': '2023-01-13T10:15:41.140623Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

We're probably most interested in `content`, which contains the JSON content of the notebook:

In [14]:
data['content']

{'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5}

For now, the notebook only has some metadata, and `cells` is empty.

### Rename a notebook

Our newly created file is still named `Untitled.ipynb`. Let's rename it to `sum.ipynb` with `PATCH /api/contents/<path>`:

In [15]:
session.patch(url + 'api/contents/Untitled.ipynb', json={'path': 'sum.ipynb'}).json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-13T10:15:41.140623Z',
 'created': '2023-01-13T10:15:43.872138Z',
 'content': None,
 'format': None,
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

Confirm that it's been renamed. `Untitled.ipynb` no longer exists:

In [16]:
session.get(url + 'api/contents/Untitled.ipynb').json()

{'message': 'No such file or directory: Untitled.ipynb', 'reason': None}

... but `sum.ipynb` does:

In [17]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-13T10:15:41.140623Z',
 'created': '2023-01-13T10:15:43.872138Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

::: {.callout-note}
You can also create a file with a specified name using `PUT /api/contents/<path>`, instead of letting the server find a unique named prefixed with `Untitled`.
:::

### Update a notebook's contents

Create a cell and append it to existing contents:

In [18]:
cell = {
    'cell_type': 'code',
    'id': '0',
    'metadata': {},
    'source': [
        '1 + 1\n',
    ],
    'outputs': [],
    'execution_count': 0,
}
data = session.get(url + 'api/contents/sum.ipynb').json()
data['content']['cells'].append(cell)

Update the notebook's contents using `PUT /api/contents/<path>`:

In [19]:
session.put(url + 'api/contents/sum.ipynb', json={'content': data['content'], 'type': 'notebook'})

<Response [200]>

Confirm that the notebook's been updated. Note that `last_modified` and `content` have both updated:

In [20]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-13T10:15:46.261982Z',
 'created': '2023-01-13T10:15:46.261982Z',
 'content': {'cells': [{'cell_type': 'code',
    'execution_count': 0,
    'id': '0',
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '1 + 1\n'}],
  'metadata': {},
  'nbformat': 4,
  'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 216,
 'writable': True,
 'type': 'notebook'}

## Executing code

Most of the functionality available inside a Jupyter Notebook in your browser is achieved by communicating with the server via websockets. This includes executing code as well as code completion.

Let's execute a very simple bit of code on the server.

### Start a session

List open sessions with `GET /api/sessions`:

In [21]:
session.get(url + 'api/sessions').json()

[]

First we need to choose a kernel specification. Here are the available options on my computer -- yours will likely differ:

In [22]:
session.get(url + 'api/kernelspecs').json()

{'default': 'python3',
 'kernelspecs': {'dyalog-kernel': {'name': 'dyalog-kernel',
   'spec': {'argv': ['python3',
     '-m',
     'dyalog_kernel',
     '-f',
     '{connection_file}'],
    'env': {},
    'display_name': 'Dyalog APL',
    'language': 'apl',
    'interrupt_mode': 'signal',
    'metadata': {}},
   'resources': {'kernel.js': '/kernelspecs/dyalog-kernel/kernel.js'}},
  'python3': {'name': 'python3',
   'spec': {'argv': ['python',
     '-m',
     'ipykernel_launcher',
     '-f',
     '{connection_file}'],
    'env': {},
    'display_name': 'Python 3 (ipykernel)',
    'language': 'python',
    'interrupt_mode': 'signal',
    'metadata': {'debugger': True}},
   'resources': {'logo-64x64': '/kernelspecs/python3/logo-64x64.png',
    'logo-32x32': '/kernelspecs/python3/logo-32x32.png',
    'logo-svg': '/kernelspecs/python3/logo-svg.svg'}}}}

Create a new session with `POST /api/sessions` with the `python3` kernelspec:

In [23]:
data = session.post(url + 'api/sessions', json={'kernel': {'name': 'python3'}, 'name': 'sum.ipynb', 'path': 'sum.ipynb', 'type': 'notebook'}).json()
data

{'id': '90e534e8-a917-481b-9939-a95a05c21b63',
 'path': 'sum.ipynb',
 'name': 'sum.ipynb',
 'type': 'notebook',
 'kernel': {'id': '935e5365-f3fc-45cf-8be1-50716cb74a5d',
  'name': 'python3',
  'last_activity': '2023-01-13T10:15:50.362933Z',
  'execution_state': 'starting',
  'connections': 0},
 'notebook': {'path': 'sum.ipynb', 'name': 'sum.ipynb'}}

Now that a session exists, we can connect to a websocket. We'll need the `kernel_id` and `session_id` to do that, so let's store them for the next step:

In [24]:
kernel_id = data['kernel']['id']
session_id = data['id']

### Communicate over WebSockets

First we need to let the server know that we'd like to make a websocket connection. This request looks quite different from the others. We include `Connection` and `Upgrade` headers:

In [25]:
session.get(url + f'api/kernels/{kernel_id}/channels?session_id={session_id}', headers={'Connection': 'upgrade', 'Upgrade': 'websocket'})

<Response [426]>

The server responds with a 426, which means that it would like to upgrade the connection -- in our case, to a websocket.

We'll use the [websockets](https://websockets.readthedocs.io/en/stable/) library to make the connection.

First, let's craft a message to request an execution:

In [46]:
import uuid

code_msg_id = str(uuid.uuid1())
code_msg = {'channel': 'shell',
            'content': {'silent': False, 'code': code},
            'header': {'msg_id': code_msg_id, 'msg_type':'execute_request'},
            'metadata': {},
            'parent_header':{}}

Now we can send the message to the server and receive all responses -- you can try changing the value of the `code` variable below to execute something else:

In [31]:
code = '1 + 1'

In [45]:
import asyncio, json, websockets, uuid

async def recv_all(websocket):
    while True:
        try: msg = json.loads(await asyncio.wait_for(websocket.recv(), 1))
        except asyncio.TimeoutError: break
        print(f"  type: {msg['msg_type']:16} content: {msg['content']}")

async def main():
    host, port = urlparse(url_with_token).netloc.split(':')
    url = f'ws://{host}:{port}/api/kernels/{kernel_id}/channels?session_id={session_id}'
    headers = {'Authorization': f'token {token}'}
    async with websockets.connect(url, extra_headers=headers) as websocket:
        print('Receiving initial messages\n')
        await recv_all(websocket)        
        print('\nSending execute_request\n')
        await websocket.send(json.dumps(code_msg))
        print('Receiving execute_reply\n')
        await recv_all(websocket)

asyncio.get_event_loop().create_task(main());

Receiving initial messages

  type: status           content: {'execution_state': 'busy'}
  type: status           content: {'execution_state': 'idle'}
  type: status           content: {'execution_state': 'idle'}

Sending execute_request

Receiving execute_reply

  type: status           content: {'execution_state': 'busy'}
  type: execute_input    content: {'code': '1 + 1', 'execution_count': 10}
  type: execute_result   content: {'data': {'text/plain': '2'}, 'metadata': {}, 'execution_count': 10}
  type: execute_reply    content: {'status': 'ok', 'execution_count': 10, 'user_expressions': {}, 'payload': []}
  type: status           content: {'execution_state': 'idle'}


Yay! We successfully executed code on the server via websockets.

You can learn more about Jupyter's messaging specification in the [Jupyter Client docs](https://jupyter-client.readthedocs.io/en/stable/messaging.html#messaging-in-jupyter).

## Cleanup

It's always good practice to cleanup after ourselves, particularly if we share the server with other users.

Let's close our session and shutdown the server (although we probably wouldn't shut it down if we shared it with others!).

### Close the session

Since we're done with the session, we can close it via `DELETE /api/sessions/<session_id>`:

In [874]:
session.delete(url + f'api/sessions/{session_id}')

<Response [204]>

### Shutdown the server

Finally, shutdown the server via `POST /api/shutdown`.

In [875]:
session.post(url + 'api/shutdown')

<Response [200]>

... and confirm that it's been shutdown correctly:

In [876]:
try: session.get(url)
except requests.exceptions.ConnectionError: print('Server has been successfully shutdown!')

Server has been successfully shutdown!


All done!

## Next steps

Congrats! If you followed all the way to the end, you've now created a barebones Jupyter frontend. Here are some directions you might consider to take this further:

- How would you implement other notebook features like code completion?
- How does Jupyter's trust system work?
- How would you implement Jupyter's checkpointing system?
- Can you redo this in another language?
- How would you design and build your own UI on top of this?

As for me, my next step is to start translating these into Swift as part of the native macOS Jupyter frontend I'm building.

Let me know on Twitter or via email if you enjoyed this or if you have any questions!